<a href="https://colab.research.google.com/github/BrajanNieto/KNN-HPC/blob/main/Proyecto_HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Computación Paralela — k-NN distribuido con MPI (mpi4py)**

---

*   Este notebook aplica conceptos de cómputo distribuido con MPI para acelerar el algoritmo **k-Nearest Neighbors (k-NN)**. El flujo implementado: genera un dataset multicategoría, distribuye el conjunto de entrenamiento entre procesos (Scatter), difunde el conjunto de prueba (Bcast), calcula distancias localmente por proceso, reúne resultados parciales (Gather) y aplica voto mayoritario para la predicción final.  
El objetivo es medir tiempos de envío, cómputo y recolección y analizar cómo escala el k-NN usando `mpi4py`. El rank 0 opcionalmente proyecta el dataset con PCA para visualizar etiquetas reales vs predicciones.


**Autores:**  

Nieto Espinoza, Brajan E.  
[brajan.nieto@utec.edu.pe](mailto:brajan.nieto@utec.edu.pe)

Tapia Chasquibol,  Mateo J.  
[mateo,.tapia@utec.edu.pe](mailto:mateo.tapia@utec.edu.pe)

<img src="https://pregrado.utec.edu.pe/sites/default/files/logo-utec-h_0_0.svg" width="190" alt="Logo UTEC" loading="lazy" typeof="foaf:Image">      

---

### 0. Librerías

In [ ]:
from mpi4py import MPI
import numpy as np
from collections import Counter
import sys


### Imports Condicionales
Solo el rank 0 importa sklearn (generación de datos y PCA) y la parte de visualización (comentada).


In [ ]:
if MPI.COMM_WORLD.Get_rank() == 0:
    from sklearn.datasets import make_classification
    from sklearn.decomposition import PCA


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Clases seleccionadas: ['0=airplane', '1=automobile', '8=ship']
Dimensiones X_train: (12000, 32, 32, 3) X_test: (3000, 32, 32, 3)
Distribución train: {'airplane': 4000, 'automobile': 4000, 'ship': 4000}
Distribución test : {'airplane': 1000, 'automobile': 1000, 'ship': 1000}
